In [ ]:
# !pip install --pre azure-containerregistry
# !pip install azure-containerregistry
# !pip install azure-identity

In [ ]:
# Create a ContainerRegistryClient that will authenticate through Active Directory
from azure.containerregistry import ContainerRegistryClient
from azure.identity import DefaultAzureCredential

endpoint = "CONTAINER_REGISTRY_NAME.azurecr.io"
audience = "https://management.azure.com"
client = ContainerRegistryClient(endpoint, DefaultAzureCredential(), audience=audience)

In [ ]:
repositories = []

In [ ]:
for repository_name in client.list_repository_names():
    # this if is optional. used to filter some specific repositories.
    if repository_name.startswith("FILTER_ONE") or repository_name.startswith("FILTER_TWO"):
        repositories.append({"name": repository_name})

In [ ]:
#print(repositories)

In [ ]:
from datetime import datetime, timedelta

def convertDateToGMT_3(data):
    """Converts a datetime object to GMT-3.
        Args:
         date: A datetime object.

        Returns:
         A new datetime object representing the date in GMT-3.
    """
    fuso_gmt_menos3 = timedelta(hours=-3)
    return data + fuso_gmt_menos3

def orderTagsByLastUpdate(tags):
    """Sorts a list of tags by last updated date in descending order.

       Args:
           tags: List of dictionaries containing tags, each with the keys 'tag_name' and 'last_update'.

       Returns:
           List of tags sorted by last updated date in descending order.
    """
    # Sort the list of tags using the 'last_update' key in descending order.
    tags_ordenadas = sorted(tags, key=lambda tag: tag['last_update'], reverse=True)    
    return tags_ordenadas

def getTagsToDelete(tags):
    """Returns all items in a list except the first two,
        with treatment for lists smaller than 3 items.

       Args:
           tags: List of items.

       Returns:
           List containing the items except the first two, or the original list if it has less than 3 items.
    """
    if len(tags) < 3:
        return []
    else:
        return tags[2:] # Returns items starting at index 2 (third item)

In [ ]:
for repository in repositories:
    manifests = client.list_tag_properties(repository["name"])
    tags = []
    for manifest in manifests:
        tags.append({'name': manifest.name, 'last_update': convertDateToGMT_3(manifest.last_updated_on)})
    repository['tags'] = orderTagsByLastUpdate(tags)

In [ ]:
#print(repositories)

In [ ]:
def deleteTag(repository_name, tag_name):
    get_manifest_result = client.get_manifest(repository_name, tag_name)
    # delete tag
    client.delete_manifest(repository_name, get_manifest_result.digest)
    print(f'tag: {repository_name}:{tag_name} deleted successfully')

In [ ]:
for repository in repositories:
    for tag in getTagsToDelete(repository['tags']):
        deleteTag(repository['name'], tag['name'])